In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from getpass import getpass
import time
import pandas as pd
from PIL import Image
import chromedriver_autoinstaller as chromedriver
chromedriver.install()
options = Options()
options.add_argument("start-maximized")
from IPython.display import display, HTML
from tabulate import tabulate,tabulate_formats
pd.set_option('display.colheader_justify', 'center')

In [2]:
a = [f"21031F00{i:02d}" for i in range(1,31)]
url = "http://jntuhsit.org/"
driver = webdriver.Chrome(options=options,executable_path = "C:\\Users\\aakas\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\101\\chromedriver.exe")
driver.get(url)
driver.find_element_by_xpath("//a[@href='http://14.139.82.140']").click()
driver.switch_to.window(driver.window_handles[1])
time.sleep(2)
All = {}
for i in a:
    driver.find_element_by_id('uid').send_keys(i)
    if i=="21031F0005":
        driver.find_element_by_id('pwd').send_keys(getpass(f"{i} Enter Your Password : "))
    else:
        driver.find_element_by_id('pwd').send_keys("sit123")
    driver.find_element_by_css_selector("input[type='submit']").click()
    if driver.find_element_by_xpath(".//div[contains(@class,'alert')]").text == '×\n  Failed !  Invalid LoginID or Password....':
        continue
    else:
        time.sleep(1)
        driver.find_element_by_xpath("//div[@id='myNavBar']/ul/li[4]").click()
        time.sleep(1)
        driver.find_element_by_xpath("//a[@href='viewResults']").click()
        time.sleep(1)
        if driver.find_element_by_xpath(".//div[contains(@class,'alert')]").text == '×\n  Failed !  Results records not found.':
            driver.find_element_by_xpath("//a[@href='logout']").click()
            continue
        else:
            driver.find_element_by_xpath("//select[@id='ecode']/option[text()='M.C.A. I Year I Semester (R20) (Reg) - April, 2022']").click()
            time.sleep(1)
            driver.find_element_by_xpath(".//button[contains(@class,'btn')]").click()
            time.sleep(1)
            Result = driver.find_element_by_xpath('//div[@id="resBlock"]//table').text.split("\n")
    All[i] = Result
    driver.find_element_by_xpath("//a[@href='logout']").click()
print(driver.find_element_by_class_name('alert-success').text.strip(".")[2:40])
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()

C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path = "C:\\Users\\aakas\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\101\\chromedriver.exe")
C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@href='http://14.139.82.140']").click()
C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('uid').send_keys(i)
C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('pwd').send_keys("sit123")
C:\Users\aa

21031F0005 Enter Your Password : ········


C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@href='logout']").click()
C:\Users\aakas\AppData\Local\Temp/ipykernel_7664/3202603049.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  print(driver.find_element_by_class_name('alert-success').text.strip(".")[2:40])


  Success !  Successfully Logged out


In [3]:
def Grade_Table(roll):
    global columns 
    columns = ["S.No","SubjectCode","SubjectName","Grade","GradePoint","Credits","Result"]
    sln = [i+1 for i in range(len(All[roll][10:-1]))]
    D = {}
    for i in columns:
        D[i] = []
    D[columns[0]] = sln
    D[columns[1]] = [All[roll][10:-1][i].split()[0] for i in range(len(All[roll][10:-1]))]
    D[columns[2]] = [' '.join(All[roll][10:-1][i].split()[1:-4]) for i in range(len(All[roll][10:-1]))]
    D[columns[3]] = [All[roll][10:-1][i].split()[-4] for i in range(len(All[roll][10:-1]))]
    D[columns[4]] = [All[roll][10:-1][i].split()[-3] for i in range(len(All[roll][10:-1]))]
    D[columns[5]] = [All[roll][10:-1][i].split()[-2] for i in range(len(All[roll][10:-1]))]
    D[columns[6]] = [All[roll][10:-1][i].split()[-1] for i in range(len(All[roll][10:-1]))]
    df = pd.DataFrame(D)
    df[['GradePoint', 'Credits']] = df[['GradePoint', 'Credits']].astype('int64')
    return df

In [85]:
# def SGPA(roll):
#     df = Grade_Table(roll)
#     df[['GradePoint', 'Credits']] = df[['GradePoint', 'Credits']].astype('int64')
#     return round(sum(df.GradePoint*df.Credits)/sum(df.Credits),2)

In [9]:
round(sum(Grade_Table('21031F0024')["GradePoint"] * Grade_Table('21031F0024')["Credits"])/sum(Grade_Table('21031F0024')["Credits"]),2)

8.38

In [12]:
All_Results = pd.DataFrame(columns=['RollNumber','Name','SGPA'])
All_Results['RollNumber'] = [list(All.keys())[i] for i in range(len(list(All.keys())))]
All_Results['Name']       = [All[list(All.keys())[i]][7][6:] for i in range(len(list(All.keys())))]
All_Results['SGPA']       = [round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2) for i in list(All.keys())]
# All_Results['SGPA']       = [SGPA(list(All.keys())[i]) for i in range(len(list(All.keys())))]
# All_Results
# print(pd.DataFrame(All_Results).to_string(index=False))
# display(tabulate(All_Results,showindex=False,headers =All_Results.columns,tablefmt=tabulate_formats[-2]))
HTML(All_Results.to_html(index=False))

RollNumber,Name,SGPA
21031F0002,DADYREDDY DHEERAJ KUMAR REDDY,7.10
21031F0004,GANGAPURAM AKHILA,8.14
21031F0005,AAKASH NADUPALLI,9.00
21031F0007,RENUKUNTLA SHIVA RAMA KRISHNA,8.14
21031F0008,VEMURI VIDYA MADHURI,9.24
21031F0009,BOLLIPALLI AKANKSHA,9.05
21031F0010,CHINTHA CHANDANA,8.19
21031F0012,KARNE RAMYA,7.62
21031F0014,M HARSHITH,7.24
21031F0015,BATTIKADI SREEDIVYA,8.57


In [40]:
# HTML(All_Results.to_html(index=False))
# All_Results.to_dict(orient='list')

In [13]:
def Check_Result(roll):
    if roll=='05':
        display(HTML("<div style='position: absolute;top: 200px; right: 80px; width: 200px; text-align:right;'><img width=80,height=70 src='Aakash_New_PassportPhoto.jpg'></div>"))
    roll = '21031F00'+roll
    l= [400,505,400,465,475,400,170,170,170]
    t =[55,88,95,115,140,185,230,250,272]
    h = ['h3','h7','h4','h4','h6','I','I','I','I']
    s = [18,13,16,16,14,17,14,14,14]
    c = ['#f7118b','#0a21a8','#0a21a8','#0a21a8','Red','Black','Black','Black','Black']
    display(HTML("<div style='position: absolute;top: 80px; left: 180px; width: 200px; text-align:left;'><img width=130,height=50 src='Jntuh logo.png'></div>"))
    for i,j,k,m,n,o in zip(range(len(All['21031F0005'][:9])),range(len(t)),range(len(l)),range(len(h)),range(len(s)),range(len(c))):
        display(HTML("<div style='position: absolute;top: {vl}px; left: {lf}px; width: 500px; text-align:left;'><{hd} style='font-family:Arial;color:{cr};font-size:{st}px;'>{Tx}</{hd}></div>".format(Tx = All[roll][:9][i],vl=t[j],lf=l[k],hd=h[m],st=s[n],cr=c[o])))    
    print('\n'*10)
#     display(tabulate(Grade_Table(roll),showindex=False,headers =columns,tablefmt=tabulate_formats[-2]))
    display(HTML(Grade_Table(roll).to_html(index=False)))
    print()
#     print(' '*50,'SGPA : ',SGPA(roll))
    print(' '*50,'SGPA : ',sum(Grade_Table('21031F0005')["GradePoint"] * Grade_Table('21031F0005')["Credits"])/sum(Grade_Table('21031F0005')["Credits"]))

In [14]:
Check_Result(input("Enter Last Two Digits of your RollNumber : "))

Enter Last Two Digits of your RollNumber : 05


S.No,SubjectCode,SubjectName,Grade,GradePoint,Credits,Result
1,MC110,MATHEMATICS FOR COMPUTER SCIENCE,A,8,3,P
2,MC120,COMPUTER ARCHITECTURE & ORGANIZATION,A+,9,3,P
3,MC130,OBJECT ORIENTED PROGRAMMING THROUGH C++,A+,9,3,P
4,MC140,OPERATING SYSTEMS,A+,9,3,P
5,MC150,DATABASE SYSTEMS,A,8,3,P
6,MC160,COMPUTER ARCHITECTURE & ORGANIZATION & OPERATING SYSTEMS LAB,O,10,2,P
7,MC170,OBJECT ORIENTED PROGRAMMING THROUGH C++ LAB,O,10,2,P
8,MC180,DATABASE SYSTEMS LAB,O,10,2,P
9,MC19A,PROFESSIONAL COMMUNICATION SKILLS,A,8,0,P



                                                   SGPA :  9.0


In [108]:
# display(pd.DataFrame(All['21031F0005'][0:6]))
# display(pd.DataFrame(All['21031F0005'][6:9]))
# display(Grade_Table('21031F0005'))
# displaypd.DataFrame(data=['SGPA',":",sum(Grade_Table('21031F0005')["GradePoint"] * Grade_Table('21031F0005')["Credits"])/sum(Grade_Table('21031F0005')["Credits"])]).transpose())

In [15]:
a =pd.DataFrame(All['21031F0005'][0:6])
b =pd.DataFrame(All['21031F0005'][6:9])
c =Grade_Table('21031F0005')
d =pd.DataFrame(data=['SGPA',sum(Grade_Table('21031F0005')["GradePoint"] * Grade_Table('21031F0005')["Credits"])/sum(Grade_Table('21031F0005')["Credits"])]).transpose()

In [31]:
# pd.concat([a,b,c,d], axis=0, ignore_index=True)

In [32]:
x = pd.read_excel('output2.xlsx')
x

,Unnamed: 0,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,(Constituent Unit of JNTUH),NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Jawaharlal Nehru Technological University Hyde...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"Kukatpally - 500 085, Telangana (India).",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,(Accredited by NAAC with 'A' Grade),NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"M.C.A. I Year I Semester (R20) (Reg) - April, ...",NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,Roll No.: 21031F0008,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Name: VEMURI VIDYA MADHURI,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,Course and Branch: M.C.A.,NaN,NaN,NaN,NaN


In [40]:
columns

['S.No',
 'SubjectCode',
 'SubjectName',
 'Grade',
 'GradePoint',
 'Credits',
 'Result']

In [52]:
x.iloc[11,1:] = columns
x.iloc[11:,1:]

,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
11,S.No,SubjectCode,SubjectName,Grade,GradePoint,Credits,Result
12,1.0,MC110,MATHEMATICS FOR COMPUTER SCIENCE,O,10.0,3.0,P
13,2.0,MC120,COMPUTER ARCHITECTURE & ORGANIZATION,O,10.0,3.0,P
14,3.0,MC130,OBJECT ORIENTED PROGRAMMING THROUGH C++,A+,9.0,3.0,P
15,4.0,MC140,OPERATING SYSTEMS,A+,9.0,3.0,P
16,5.0,MC150,DATABASE SYSTEMS,A,8.0,3.0,P
17,6.0,MC160,COMPUTER ARCHITECTURE & ORGANIZATION & OPERATI...,O,10.0,2.0,P
18,7.0,MC170,OBJECT ORIENTED PROGRAMMING THROUGH C++ LAB,A+,9.0,2.0,P
19,8.0,MC180,DATABASE SYSTEMS LAB,A+,9.0,2.0,P
20,9.0,MC19A,PROFESSIONAL COMMUNICATION SKILLS,A,8.0,0.0,P


In [54]:
x.to_excel("o.xlsx",index=False,)

,Unnamed: 0,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,(Constituent Unit of JNTUH),NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Jawaharlal Nehru Technological University Hyde...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"Kukatpally - 500 085, Telangana (India).",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,(Accredited by NAAC with 'A' Grade),NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"M.C.A. I Year I Semester (R20) (Reg) - April, ...",NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,Roll No.: 21031F0008,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Name: VEMURI VIDYA MADHURI,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,Course and Branch: M.C.A.,NaN,NaN,NaN,NaN


In [165]:
# display(pd.DataFrame(data=['SGPA',":",sum(Grade_Table('21031F0005')["GradePoint"] * Grade_Table('21031F0005')["Credits"])/sum(Grade_Table('21031F0005')["Credits"])]).transpose())

In [17]:
dflist = [a,b,c,d]

In [22]:
Ex['21031F0002']

[                          0                         
 0                   SCHOOL OF INFORMATION TECHNOLOGY
 1                        (Constituent Unit of JNTUH)
 2  Jawaharlal Nehru Technological University Hyde...
 3           Kukatpally - 500 085, Telangana (India).
 4                (Accredited by NAAC with 'A' Grade)
 5  M.C.A. I Year I Semester (R20) (Reg) - April, ...,
                    0                  
 0                 Roll No.: 21031F0002
 1  Name: DADYREDDY DHEERAJ KUMAR REDDY
 2            Course and Branch: M.C.A.,
    S.No SubjectCode                    SubjectName                     Grade  \
 0    1      MC110                     MATHEMATICS FOR COMPUTER SCIENCE   B+    
 1    2      MC120                 COMPUTER ARCHITECTURE & ORGANIZATION   B+    
 2    3      MC130              OBJECT ORIENTED PROGRAMMING THROUGH C++   B+    
 3    4      MC140                                    OPERATING SYSTEMS   B+    
 4    5      MC150                                     

In [25]:
# for i,df in enumerate (dflist):
#     print(i,df)
for i in list(All.keys()):
    for j,df in enumerate (Ex[i]):
        print(i,Ex[i])

21031F0002 [                          0                         
0                   SCHOOL OF INFORMATION TECHNOLOGY
1                        (Constituent Unit of JNTUH)
2  Jawaharlal Nehru Technological University Hyde...
3           Kukatpally - 500 085, Telangana (India).
4                (Accredited by NAAC with 'A' Grade)
5  M.C.A. I Year I Semester (R20) (Reg) - April, ...,                    0                  
0                 Roll No.: 21031F0002
1  Name: DADYREDDY DHEERAJ KUMAR REDDY
2            Course and Branch: M.C.A.,    S.No SubjectCode                    SubjectName                     Grade  \
0    1      MC110                     MATHEMATICS FOR COMPUTER SCIENCE   B+    
1    2      MC120                 COMPUTER ARCHITECTURE & ORGANIZATION   B+    
2    3      MC130              OBJECT ORIENTED PROGRAMMING THROUGH C++   B+    
3    4      MC140                                    OPERATING SYSTEMS   B+    
4    5      MC150                                     DATAB

0  SGPA  8.380952]
21031F0024 [                          0                         
0                   SCHOOL OF INFORMATION TECHNOLOGY
1                        (Constituent Unit of JNTUH)
2  Jawaharlal Nehru Technological University Hyde...
3           Kukatpally - 500 085, Telangana (India).
4                (Accredited by NAAC with 'A' Grade)
5  M.C.A. I Year I Semester (R20) (Reg) - April, ...,               0             
0       Roll No.: 21031F0024
1     Name: MACHA SAIKRISHNA
2  Course and Branch: M.C.A.,    S.No SubjectCode                    SubjectName                     Grade  \
0    1      MC110                     MATHEMATICS FOR COMPUTER SCIENCE   A+    
1    2      MC120                 COMPUTER ARCHITECTURE & ORGANIZATION    A    
2    3      MC130              OBJECT ORIENTED PROGRAMMING THROUGH C++    A    
3    4      MC140                                    OPERATING SYSTEMS   B+    
4    5      MC150                                     DATABASE SYSTEMS    A    


In [26]:
pd.DataFrame(Ex['21031F0002'])

C:\Users\aakas\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
0,0 ...
1,0 0 ...
2,S.No SubjectCode Subject...
3,0 1 0 SGPA 7.095238


In [166]:
# We'll define an Excel writer object and the target file
Excelwriter = pd.ExcelWriter("MCA 1st Sem Results.xlsx",engine="xlsxwriter")

#We now we'll loop the list of dataframes
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1),index=False)
#And finally we save the file
Excelwriter.save()

In [21]:
Ex = {}
for i in list(All.keys()):
    k =pd.DataFrame(All[i][0:6])
    l =pd.DataFrame(All[i][6:9])
    m =Grade_Table(i)
    n =pd.DataFrame(data=['SGPA',sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"])]).transpose()
    dfs = [k,l,m,n]
    Ex[i] = dfs

In [ ]:
# We'll define an Excel writer object and the target file
Excelwriter = pd.ExcelWriter("MCA 1st Sem Results.xlsx",engine="xlsxwriter")
a =pd.DataFrame(All[roll][0:6])
b =pd.DataFrame(All[roll][6:9])
c =Grade_Table(roll)
d =pd.DataFrame(data=['SGPA',round(sum(Grade_Table(roll)["GradePoint"] * Grade_Table(roll)["Credits"])/sum(Grade_Table(roll)["Credits"]),2)]).transpose()
#We now we'll loop the list of dataframes
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1),index=False)
#And finally we save the file
Excelwriter.save()

In [329]:
def Send_To_Excel(roll):
    a =pd.DataFrame(All[roll][0:6])
    b =pd.DataFrame(All[roll][6:9])
    c =Grade_Table(roll)
    d =pd.DataFrame(data=['SGPA',round(sum(Grade_Table(roll)["GradePoint"] * Grade_Table(roll)["Credits"])/sum(Grade_Table(roll)["Credits"]),2)]).transpose()
    dfs = [a,b,c,d]
    startrow = 0
    startcol = [2,3,1,3]
    with pd.ExcelWriter('output2.xlsx',engine="xlsxwriter") as writer:
        for (df,c) in zip(dfs,startcol):
            df.to_excel(writer,sheet_name=roll,startcol=c,startrow=startrow,index=False,header=False)
            startrow += (df.shape[0] + 2)
    writer.save()

In [371]:
ndf = pd.read_excel('output2.xlsx')
ndf.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'SCHOOL OF INFORMATION TECHNOLOGY',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')

In [382]:
ndf.iloc[12:21,1:7]

,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
12,1.0,MC110,MATHEMATICS FOR COMPUTER SCIENCE,O,10.0,3.0
13,2.0,MC120,COMPUTER ARCHITECTURE & ORGANIZATION,O,10.0,3.0
14,3.0,MC130,OBJECT ORIENTED PROGRAMMING THROUGH C++,A+,9.0,3.0
15,4.0,MC140,OPERATING SYSTEMS,A+,9.0,3.0
16,5.0,MC150,DATABASE SYSTEMS,A,8.0,3.0
17,6.0,MC160,COMPUTER ARCHITECTURE & ORGANIZATION & OPERATI...,O,10.0,2.0
18,7.0,MC170,OBJECT ORIENTED PROGRAMMING THROUGH C++ LAB,A+,9.0,2.0
19,8.0,MC180,DATABASE SYSTEMS LAB,A+,9.0,2.0
20,9.0,MC19A,PROFESSIONAL COMMUNICATION SKILLS,A,8.0,0.0


In [396]:
del(df)

In [403]:
# for i in range(len(list(All.keys))):
df = ndf
ndf.iloc[7:10,3:4] = display(HTML(a.to_html(index=False)))
ndf.iloc[:5,2:3] = HTML(b.to_html(index=False))
# ndf.iloc[12:21,1:7] = c
# ndf.iloc[23:,3:5] = d
df

0
SCHOOL OF INFORMATION TECHNOLOGY
(Constituent Unit of JNTUH)
Jawaharlal Nehru Technological University Hyderabad
"Kukatpally - 500 085, Telangana (India)."
(Accredited by NAAC with 'A' Grade)
"M.C.A. I Year I Semester (R20) (Reg) - April, 2022"


,Unnamed: 0,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,<IPython.core.display.HTML object>,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,<IPython.core.display.HTML object>,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,<IPython.core.display.HTML object>,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,<IPython.core.display.HTML object>,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,<IPython.core.display.HTML object>,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [399]:
# HTML(a.to_html(index=False))
# ['SCHOOL OF INFORMATION TECHNOLOGY']
# ['Unnamed: 3']
a[0]

0                     SCHOOL OF INFORMATION TECHNOLOGY
1                          (Constituent Unit of JNTUH)
2    Jawaharlal Nehru Technological University Hyde...
3             Kukatpally - 500 085, Telangana (India).
4                  (Accredited by NAAC with 'A' Grade)
5    M.C.A. I Year I Semester (R20) (Reg) - April, ...
Name: 0, dtype: object

In [330]:
Send_To_Excel('21031F0005')
Send_To_Excel('21031F0008')

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [333]:
with pd.ExcelWriter('output2.xlsx',mode='a',engine='openpyxl') as writer:
    for i in list(All.keys())[1:]:
        Send_To_Excel(i)
    writer.save()

In [314]:
for i in range(len(dfs)):
    print(dfs[i])

                          0                         
0                   SCHOOL OF INFORMATION TECHNOLOGY
1                        (Constituent Unit of JNTUH)
2  Jawaharlal Nehru Technological University Hyde...
3           Kukatpally - 500 085, Telangana (India).
4                (Accredited by NAAC with 'A' Grade)
5  M.C.A. I Year I Semester (R20) (Reg) - April, ...
              0             
0       Roll No.: 21031F0005
1     Name: AAKASH NADUPALLI
2  Course and Branch: M.C.A.
3
    0    1  
0  SGPA  9.0


In [285]:
Send_To_Excel('21031F0015')

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [60]:
x = pd.DataFrame(columns=[i for i in range(1,7)])

In [61]:
x.append(Ex['21031F0005'],ignore_index=True)

,0,1,2,3,4,5,6,Credits,Grade,GradePoint,Result,S.No,SubjectCode,SubjectName
0,SCHOOL OF INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(Constituent Unit of JNTUH),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jawaharlal Nehru Technological University Hyde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Kukatpally - 500 085, Telangana (India).",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(Accredited by NAAC with 'A' Grade),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"M.C.A. I Year I Semester (R20) (Reg) - April, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Roll No.: 21031F0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Name: AAKASH NADUPALLI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Course and Branch: M.C.A.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,A,8.0,P,1.0,MC110,MATHEMATICS FOR COMPUTER SCIENCE


In [58]:
Ex['21031F0005']

[                          0                         
 0                   SCHOOL OF INFORMATION TECHNOLOGY
 1                        (Constituent Unit of JNTUH)
 2  Jawaharlal Nehru Technological University Hyde...
 3           Kukatpally - 500 085, Telangana (India).
 4                (Accredited by NAAC with 'A' Grade)
 5  M.C.A. I Year I Semester (R20) (Reg) - April, ...,
               0             
 0       Roll No.: 21031F0005
 1     Name: AAKASH NADUPALLI
 2  Course and Branch: M.C.A.,
    S.No SubjectCode                    SubjectName                     Grade  \
 0    1      MC110                     MATHEMATICS FOR COMPUTER SCIENCE    A    
 1    2      MC120                 COMPUTER ARCHITECTURE & ORGANIZATION   A+    
 2    3      MC130              OBJECT ORIENTED PROGRAMMING THROUGH C++   A+    
 3    4      MC140                                    OPERATING SYSTEMS   A+    
 4    5      MC150                                     DATABASE SYSTEMS    A    
 5    6      M

In [267]:
for i in list(All.keys()[1:]):
    with pd.ExcelWriter("path_to_file.xlsx", mode="a", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Sheet3")
    Send_To_Excel(i)

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [253]:
pd.read_excel('output2.xlsx',sheet_name='21031F0029')

,Unnamed: 0,Unnamed: 1,SCHOOL OF INFORMATION TECHNOLOGY,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,(Constituent Unit of JNTUH),NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Jawaharlal Nehru Technological University Hyde...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"Kukatpally - 500 085, Telangana (India).",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,(Accredited by NAAC with 'A' Grade),NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"M.C.A. I Year I Semester (R20) (Reg) - April, ...",NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,Roll No.: 21031F0029,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Name: JATAVATH SINDHU,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,Course and Branch: M.C.A.,NaN,NaN,NaN,NaN


In [154]:
dfs = [a,b,c,d]
startrow = 0
startcol = [2,3,1,3]
Excelwriter = pd.ExcelWriter("output1.xlsx",engine="xlsxwriter")
for (df,c) in zip(dfs,startcol):
    df.to_excel(Excelwriter,sheet_name='21031F0005',startcol=c,startrow=startrow,index=False,header=False)
    startrow += (df.shape[0] + 2)

AttributeError: 'int' object has no attribute 'to_excel'

In [137]:
print(list(All.keys()),end='')

['21031F0002', '21031F0004', '21031F0005', '21031F0007', '21031F0008', '21031F0009', '21031F0010', '21031F0012', '21031F0014', '21031F0015', '21031F0018', '21031F0021', '21031F0022', '21031F0024', '21031F0025', '21031F0029']

In [102]:
roll = '21031F0005'
a =pd.DataFrame(All[roll][0:6])
b =pd.DataFrame(All[roll][6:9])
c =pd.DataFrame(columns).transpose()
d =Grade_Table(roll)
e =pd.DataFrame(data=['SGPA',round(sum(Grade_Table(roll)["GradePoint"] * Grade_Table(roll)["Credits"])/sum(Grade_Table(roll)["Credits"]),2)]).transpose()

In [103]:
dfs = [a,b,c,d,e]
startrow = 1
startcol = [2,3,1,1,3]
with pd.ExcelWriter('output.xlsx',engine="xlsxwriter") as writer:
    for (df,c,k) in zip(dfs,startcol,range(len(dfs))):
        if k==2:
            df.to_excel(writer,sheet_name='21031F0005',startcol=c,startrow=startrow,index=False,header=False)
            startrow += 1
        else:
            df.to_excel(writer,sheet_name='21031F0005',startcol=c,startrow=startrow,index=False,header=False)
            startrow += (df.shape[0] + 2)    

In [119]:
x = pd.read_excel('output.xlsx')
x

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,SCHOOL OF INFORMATION TECHNOLOGY,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,(Constituent Unit of JNTUH),NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Jawaharlal Nehru Technological University Hyde...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"Kukatpally - 500 085, Telangana (India).",NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,(Accredited by NAAC with 'A' Grade),NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,"M.C.A. I Year I Semester (R20) (Reg) - April, ...",NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Roll No.: 21031F0005,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,Name: AAKASH NADUPALLI,NaN,NaN,NaN,NaN


In [176]:
Ex['21031F0002'][2]

,S.No,SubjectCode,SubjectName,Grade,GradePoint,Credits,Result
0,1,MC110,MATHEMATICS FOR COMPUTER SCIENCE,B+,7,3,P
1,2,MC120,COMPUTER ARCHITECTURE & ORGANIZATION,B+,7,3,P
2,3,MC130,OBJECT ORIENTED PROGRAMMING THROUGH C++,B+,7,3,P
3,4,MC140,OPERATING SYSTEMS,B+,7,3,P
4,5,MC150,DATABASE SYSTEMS,C,5,3,P
5,6,MC160,COMPUTER ARCHITECTURE & ORGANIZATION & OPERATI...,A+,9,2,P
6,7,MC170,OBJECT ORIENTED PROGRAMMING THROUGH C++ LAB,A,8,2,P
7,8,MC180,DATABASE SYSTEMS LAB,A,8,2,P
8,9,MC19A,PROFESSIONAL COMMUNICATION SKILLS,B+,7,0,P


In [186]:
x1 =x

In [187]:
x1.iloc[0:6,2:3]   =  Ex['21031F0007'][0]
x1.iloc[8:11,3:4]  =  Ex['21031F0007'][1]
x1.iloc[14:23,1:8] =  Ex['21031F0007'][2]
x1.iloc[24:,3:5]   =  Ex['21031F0007'][3]

In [183]:
x.to_excel('output.xlsx',index=False,header=False,startrow=1,sheet_name='21031F0005')

In [189]:
# x1.to_excel('output.xlsx',index=False,header=False,startrow=1,sheet_name='21031F0007')

PermissionError: [Errno 13] Permission denied: 'output.xlsx'

In [182]:
x.iloc[24:,3:5]

,Unnamed: 3,Unnamed: 4
24,NaN,NaN
25,SGPA,9.0


In [100]:
startrow = 1
for (df,c,k) in zip(dfs,startcol,range(len(dfs))):
        if k==2:
            startrow += 1
            print(startrow)
        else:
            startrow += (df.shape[0] + 2)    
            print(startrow)

9
14
15
26
29


In [74]:
for i in list(All.keys()):
    k =pd.DataFrame(All[i][0:6])
    l =pd.DataFrame(All[i][6:9])
    m =Grade_Table(i)
    n =pd.DataFrame(data=['SGPA',sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"])]).transpose()
    dfs = [k,l,m,n]
    startrow = 0
    startcol = [2,3,1,3]
    with pd.ExcelWriter('output.xlsx',engine="xlsxwriter") as writer:
        for (df,c) in zip(dfs,startcol):
            df.to_excel(writer,sheet_name=i,startcol=c,startrow=startrow,index=False,header=False)
            startrow += (df.shape[0] + 2)
    writer.save()

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [157]:
Ex = {}
for i in list(All.keys()):
    k =pd.DataFrame(All[i][0:6])
    l =pd.DataFrame(All[i][6:9])
    m =Grade_Table(i)
    n =pd.DataFrame(data=['SGPA',round(sum(Grade_Table(i)["GradePoint"] * Grade_Table(i)["Credits"])/sum(Grade_Table(i)["Credits"]),2)]).transpose()
    dfs = [k,l,m,n]
    Ex[i] = dfs

In [203]:
list(All.keys())

['21031F0002',
 '21031F0004',
 '21031F0005',
 '21031F0007',
 '21031F0008',
 '21031F0009',
 '21031F0010',
 '21031F0012',
 '21031F0014',
 '21031F0015',
 '21031F0018',
 '21031F0021',
 '21031F0022',
 '21031F0024',
 '21031F0025',
 '21031F0029']

In [ ]:
Ex['2']

In [206]:
for i in list(All.keys()):
    startrow = 0
    startcol = [2,3,1,3]
    with pd.ExcelWriter('output.xlsx',engine="xlsxwriter") as writer:
        for (df,c) in zip(range(len(Ex[i])),startcol):
            Ex[i][df].to_excel(writer,sheet_name=i,startcol=startcol[c],startrow=startrow,index=False,header=False)
            startrow += (Ex[i][df].shape[0] + 2)
        writer.save()
        continue

C:\Users\aakas\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
# We'll define an Excel writer object and the target file
Excelwriter = pd.ExcelWriter("MCA 1st Sem Results.xlsx",engine="xlsxwriter")

#We now we'll loop the list of dataframes
for i, df in enumerate (dflist):
    df.to_excel(Excelwriter, sheet_name="Sheet" + str(i+1),index=False)
#And finally we save the file
Excelwriter.save()

In [248]:
Ex['21031F0002']

[[                          0                         
  0                   SCHOOL OF INFORMATION TECHNOLOGY
  1                        (Constituent Unit of JNTUH)
  2  Jawaharlal Nehru Technological University Hyde...
  3           Kukatpally - 500 085, Telangana (India).
  4                (Accredited by NAAC with 'A' Grade)
  5  M.C.A. I Year I Semester (R20) (Reg) - April, ...,
                     0                  
  0                 Roll No.: 21031F0002
  1  Name: DADYREDDY DHEERAJ KUMAR REDDY
  2            Course and Branch: M.C.A.,
     S.No SubjectCode                    SubjectName                     Grade  \
  0    1      MC110                     MATHEMATICS FOR COMPUTER SCIENCE   B+    
  1    2      MC120                 COMPUTER ARCHITECTURE & ORGANIZATION   B+    
  2    3      MC130              OBJECT ORIENTED PROGRAMMING THROUGH C++   B+    
  3    4      MC140                                    OPERATING SYSTEMS   B+    
  4    5      MC150                    